In [ ]:
# ------------------------------- IMPORT MODULES & SETUP ------------------------------------------------
# Standard Libraries
import os
# os.environ['OMP_NUM_THREADS'] = '48'
import math
import json
import datetime
import numpy as np
import time
# import nvtx

import argparse

# ------------------------------- CUSTOM FUNCTIONS ------------------------------------------------
# Custom Library
import sys
sys.path.append('../')

from proxy_apps import ProxyTSPRD
# from proxy_apps.apps.timeseries_prediction import deepDMD, proxyDeepDMD, proxyDeepDMDMGPU, proxyDeepDMDPyTorch, proxyDeepDMDPyTorchJIT, hyperparameters

# from proxy_apps.utils.tf import TimingCallback
# from proxy_apps.utils.data.main import NpEncoder
# from proxy_apps.utils import file_reader, path_handler
# from proxy_apps.utils.data.grid import GridNetworkDataHandler, GridNetworkTFDataHandler, GridNetworkNewGen, GridDataGenPyTorch

: 

In [1]:
import torch

In [3]:
import time
start_time = time.time()
torch.tensor(start_time)

tensor(1.6372e+09)

In [2]:
import numpy as np
test_data = np.random.rand(32, 60, 136)

In [3]:
import tensorflow as tf
print("TensorFlow Version: ", tf.__version__)
class TFLSTM(tf.keras.Model):
    def __init__(self, bw_size, fw_size, n_features):
        super(TFLSTM, self).__init__(name = 'TFLSTM')
        self.lstm1_layer   = tf.keras.layers.LSTM(512, input_shape=(bw_size, n_features), return_sequences=True)
        self.lstm2_layer   = tf.keras.layers.LSTM(256, return_sequences=True)
        self.lstm3_layer   = tf.keras.layers.LSTM(128, return_sequences=True)
        self.lstm4_layer   = tf.keras.layers.LSTM(64)
        self.dense_layer   = tf.keras.layers.Dense(fw_size * n_features)
        self.output_layer  = tf.keras.layers.Reshape((fw_size, n_features))
        
    def call(self, input_data):
        print(input_data.shape)
        fx = self.lstm1_layer(input_data)        
        print(fx.shape)
        fx = self.lstm2_layer(fx)        
        print(fx.shape)
        fx = self.lstm3_layer(fx)        
        print(fx.shape)
        fx = self.lstm4_layer(fx)        
        print(fx.shape)
        fx = self.dense_layer(fx)        
        print(fx.shape)
        return self.output_layer(fx)

TensorFlow Version:  2.4.1


In [4]:
tf_model=TFLSTM(60, 30, 136)
tf_model.build(input_shape=(None, 60, 136))
tf_model.summary()

(None, 60, 136)


2021-11-05 19:17:40.353570: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-11-05 19:17:40.353776: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64/:/share/apps/python/miniconda3.9/lib
2021-11-05 19:17:40.353788: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-11-05 19:17:40.353812: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (deception02.pnl.gov): /proc/driver/nvidia/version does not exist
2021-11-05 19:17:40.354060: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AV

(None, 60, 512)
(None, 60, 256)
(None, 60, 128)
(None, 64)
(None, 4080)
Model: "TFLSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  multiple                  1329152   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  787456    
_________________________________________________________________
lstm_2 (LSTM)                multiple                  197120    
_________________________________________________________________
lstm_3 (LSTM)                multiple                  49408     
_________________________________________________________________
dense (Dense)                multiple                  265200    
_________________________________________________________________
reshape (Reshape)            multiple                  0         
Total params: 2,628,336
Trainable params: 2,628,336
No

In [5]:
tf_out = tf_model(test_data)

(32, 60, 136)
(32, 60, 512)
(32, 60, 256)
(32, 60, 128)
(32, 64)
(32, 4080)


In [30]:
import torch
from torch.autograd import Variable
class PTLSTM(torch.nn.Module):
    def __init__(self, bw_size, fw_size, n_features):
        super(PTLSTM, self).__init__()
        self.bw_size = bw_size # size of the backward window
        self.fw_size = fw_size # size of the backward window
        self.n_features = n_features # size of the backward window
        
        self.hidden_size1 = 512
        self.lstm1_layer   = torch.nn.LSTM(hidden_size=self.hidden_size1, input_size=self.n_features, batch_first=True)
        self.hidden_size2 = 256
        self.lstm2_layer   = torch.nn.LSTM(hidden_size=self.hidden_size2, input_size=self.hidden_size1, batch_first=True)
        self.hidden_size3 = 128
        self.lstm3_layer   = torch.nn.LSTM(hidden_size=self.hidden_size3, input_size=self.hidden_size2, batch_first=True)
        self.hidden_size4 = 64
        self.lstm4_layer   = torch.nn.LSTM(hidden_size=self.hidden_size4, input_size=self.hidden_size3, batch_first=True)
        self.dense_layer   = torch.nn.Linear(in_features=self.hidden_size4, out_features=fw_size * n_features)
        # self.output_layer  = tf.keras.layers.Reshape((fw_size, n_features))
    
    def forward(self,x):
        print(x.shape)
        h_1 = Variable(torch.zeros(1, x.size(0), self.hidden_size1)) #hidden state
        c_1 = Variable(torch.zeros(1, x.size(0), self.hidden_size1)) #internal state
        h_1, c_1 = self.lstm1_layer(x, (h_1, c_1))
        #print(h_1.shape)
        
        h_2 = Variable(torch.zeros(1, x.size(0), self.hidden_size2)) #hidden state
        c_2 = Variable(torch.zeros(1, x.size(0), self.hidden_size2)) #internal state
        h_2, c_2 = self.lstm2_layer(h_1, (h_2, c_2)) #first Dense
        #print(h_2.shape)
        
        h_3 = Variable(torch.zeros(1, x.size(0), self.hidden_size3)) #hidden state
        c_3 = Variable(torch.zeros(1, x.size(0), self.hidden_size3)) #internal state
        h_3, c_3 = self.lstm3_layer(h_2, (h_3, c_3)) #relu
        #print(h_3.shape)
        
        h_4 = Variable(torch.zeros(1, x.size(0), self.hidden_size4)) #hidden state
        c_4 = Variable(torch.zeros(1, x.size(0), self.hidden_size4)) #internal state
        h_4, c_4 = self.lstm4_layer(h_3, (h_4, c_4)) #Final Output
        #print(h_4[:, -1, :].shape)
        
        out = self.dense_layer(h_4[:, -1, :])
        #print(out.shape)
        
        return out.view((out.shape[0], self.fw_size, self.n_features))

In [31]:
pt_model = PTLSTM(60, 30, 136)
pt_model

PTLSTM(
  (lstm1_layer): LSTM(136, 512, batch_first=True)
  (lstm2_layer): LSTM(512, 256, batch_first=True)
  (lstm3_layer): LSTM(256, 128, batch_first=True)
  (lstm4_layer): LSTM(128, 64, batch_first=True)
  (dense_layer): Linear(in_features=64, out_features=4080, bias=True)
)

In [29]:
pt_out = pt_model(torch.Tensor(test_data))

torch.Size([32, 60, 136])


NameError: name 'h1' is not defined

In [26]:
tf_out.numpy()[0][0, :] - pt_out.detach().numpy()[0][0, :]

array([ 0.00070119, -0.05698129, -0.00950519,  0.07187919,  0.08011659,
       -0.1108347 , -0.02103576,  0.03646854, -0.00913914,  0.03987632,
       -0.09225996,  0.04328468, -0.02160279, -0.06178767,  0.14740677,
        0.06635267,  0.07306971,  0.01730229, -0.07763304, -0.0875956 ,
       -0.10194558,  0.02492072,  0.08379532,  0.03210932,  0.0082323 ,
        0.00502864, -0.04964275,  0.02318013,  0.12815316,  0.0258281 ,
       -0.00368786,  0.12309135,  0.06032683, -0.08208665,  0.05897676,
       -0.02617712,  0.01351137, -0.08481851, -0.13048518,  0.01454673,
       -0.05493959, -0.0502141 , -0.03074871, -0.02350655,  0.00768772,
        0.10149132, -0.08347325,  0.00068326,  0.1552209 ,  0.12592465,
        0.00596165, -0.01254958, -0.0046465 , -0.00265353,  0.07255635,
        0.039815  , -0.12123776, -0.07699578,  0.01123032, -0.13657816,
       -0.05917638,  0.08512174,  0.01199227,  0.06013994,  0.15003107,
        0.10146493,  0.03452443, -0.11632743, -0.06067556, -0.04

In [14]:
pt_out

tensor([[[ 0.0045,  0.0393,  0.0057,  ...,  0.0129, -0.0080, -0.0459],
         [-0.0321,  0.0591,  0.0067,  ..., -0.0347, -0.0763,  0.0437],
         [ 0.1468,  0.0329, -0.1611,  ...,  0.0254, -0.1120, -0.1399],
         ...,
         [ 0.0253, -0.0378,  0.0088,  ..., -0.0833,  0.0256,  0.0148],
         [-0.0577, -0.0450,  0.0125,  ..., -0.0560,  0.0115,  0.1055],
         [ 0.0697, -0.0556,  0.0870,  ...,  0.0130,  0.1178,  0.1172]],

        [[ 0.0043,  0.0391,  0.0049,  ...,  0.0128, -0.0080, -0.0460],
         [-0.0342,  0.0582,  0.0071,  ..., -0.0358, -0.0775,  0.0437],
         [ 0.1477,  0.0326, -0.1608,  ...,  0.0256, -0.1115, -0.1402],
         ...,
         [ 0.0244, -0.0379,  0.0084,  ..., -0.0834,  0.0261,  0.0160],
         [-0.0585, -0.0443,  0.0122,  ..., -0.0568,  0.0112,  0.1056],
         [ 0.0699, -0.0552,  0.0878,  ...,  0.0124,  0.1183,  0.1167]],

        [[ 0.0042,  0.0403,  0.0056,  ...,  0.0127, -0.0080, -0.0459],
         [-0.0320,  0.0594,  0.0060,  ..., -0

In [ ]:
## Initialize ProxyTSPRD
# model info
_MODEL_NAME = "TFDataOptMGPU"
_N_EPOCHS = 2
_BATCH_SIZE = 32
_MIXED_PRECISION = False

# machine info
_MACHINE_NAME = 'A100'
_N_GPUS = 1
_N_CPUS = 16

# paths
_CURR_DIR = os.path.dirname('./')
_DATA_DIR = _CURR_DIR
_MODEL_DIR = _CURR_DIR

_FRAMEWORK = 'TF'

interface = ProxyTSPRD('config.json', _MODEL_NAME, _N_EPOCHS, _BATCH_SIZE, _MIXED_PRECISION, 
                 _MACHINE_NAME, _N_GPUS, _N_CPUS,
                 _DATA_DIR, _MODEL_DIR, _FRAMEWORK)

In [ ]:
interface.load_data(_CURR_DIR)

In [ ]:
interface.train_model()